In [43]:
#Generate plot 3 showing the efect of moments on the temp distribution.

In [44]:
import numpy as np
import xarray as xr
from distributed import Client
import dask_jobqueue
#import dask
#import daskdataframe as dd
import matplotlib.pyplot as plt
import matplotlib as mtplt
import glob
import netCDF4 as nc
import zarr
import nc_time_axis
#import xskillscore as xs
import cartopy as cart
#%run data_handler.ipynb
#np.set_printoptions(threshold=np.inf)
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from scipy.special import erfinv, erf
##mtplt.use("TkCairo")
import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

In [45]:
def get_data_array(filename, recalculate=False):
    """
    Fetches an xarray data array from a zarr file. Recalculates if the file doesn't exist or if recalculate is True.

    Parameters:
    -----------
    filename: str
        The path to the zarr file.
    recalculate: bool
        If True, recalculate the data array. If False (default), load the data array from the file if it exists.

    Returns:
    --------
    xr.DataArray
        The data array.
    """

    if recalculate or not zarr.is_zarr(filename):
        # Recalculate the data array
        # TODO: replace this with your own code to calculate the data array
        data_array = xr.DataArray([[1, 2], [3, 4]], dims=('x', 'y'))
        
        # Save the data array to the zarr file
        data_array.to_dataset(name='data_array').to_zarr(filename)
    else:
        # Load the data array from the zarr file
        data_array = xr.open_zarr(filename)['data_array']

    return data_array

In [46]:
def to_daily(ds):
    year = ds.time.dt.year
    day = ds.time.dt.dayofyear

    # assign new coords
    ds = ds.assign_coords(year=("time", year.data), day=("time", day.data))

    # reshape the array to (..., "day", "year")
    return ds.set_index(time=("year", "day")).unstack("time")  

In [47]:
def stacker(x,stackdim_name):
    #Assumes that x has dimensions year and member
    #x = x.stack(stackdim_name=('year','member'),create_index=False)
    x = x.stack({stackdim_name: ('year', 'member')},create_index=False)
    return x

In [48]:
def stack_roll(ds,roll_len,stride_len,stackdim_name):
    #Assumes that ds has dimensions year and member
    ds_stack   = stacker(ds,stackdim_name)
    #print(ds_stack)
    ds_roll    = ds_stack.rolling({stackdim_name:roll_len}, center=True).construct('index', stride=stride_len)
    
    return ds_roll 

In [49]:
def mean_adj(ds_stack,ds_mroll):
    ds = (ds_stack - ds_stack.mean(dim='myear')) + ds_mroll
    return ds

def mean_std_adj(ds_stack,ds_mroll,ds_sroll):
    ds = (ds_stack -ds_stack.mean(dim='myear'))*(ds_sroll/ds_stack.std(dim='myear'))\
          + ds_mroll
    return ds

In [50]:
def gauss_quantile(mean,std,quantile):
    qvalue = mean + std * np.sqrt(2) * erfinv(2*quantile-1)
    return qvalue

In [51]:
def altspace(start, step, count, endpoint=False, **kwargs):
   stop = start+(step*count)
   return np.linspace(start, stop, count, endpoint=endpoint, **kwargs)

In [52]:
# #Chicago
LAT        = 41.8781
LON        = (360-87.6298)%360
# # #Bengaluru:
# LAT        = 12.9716
# LON        = 77.5946
######################
chic_lat  = 41.8781
chic_lon  = (360-87.6298)%360
ben_lat   = 12.9716
ben_lon   = 77.5946
########## CONUS ############
#CONUS lat-lon
top       = 50.0 # north lat
left      = -124.7844079+360 # west long
right     = -66.9513812+360 # east long
bottom    =  24.7433195 # south lat
################################
savefigs   = '/global/scratch/users/harsha/savefigs/Feb21/'
cesm2_path = '/global/scratch/users/harsha/LENS/cesm2/tasmax/'
cvals      = '/global/scratch/users/harsha/LENS/cesm2/cvals/'
cvals_det  = '/global/scratch/users/harsha/LENS/cesm2/cvals/detrended/'
loc        = 'Chicago/'
locn       = 'Chicago'
cvals1     = '/global/scratch/users/harsha/LENS/cesm2/tmax_mem'
###########
print('(lat,lon)=',LAT,LON)

(lat,lon)= 41.8781 272.3702


In [53]:
job_extra = ['--qos=cf_lowprio','--account=ac_cumulus'] 
#job_extra =['--qos=lr6_lowprio','--account=ac_cumulus','--constraint=lr6_m192']
#job_extra =['--qos=condo_cumulus_lr6','--account=lr_cumulus','--constraint=lr6_m192']
#job_extra =['--qos=lr_lowprio','--account=ac_cumulus']
cluster = dask_jobqueue.SLURMCluster(queue="cf1", cores=20, walltime='5:00:00', 
                local_directory='/global/scratch/users/harsha/dask_space/', 
                log_directory='/global/scratch/users/harsha/dask_space/', 
                job_extra_directives=job_extra, interface='eth0', memory="192GB") 
client  = Client(cluster) 
cluster.scale_up(3)

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42641 instead
  warnings.warn(


In [54]:
cluster

In [55]:
pi_year0 = '1850'
pi_year1 = '1879'
ic_year0 = '2071'
ic_year1 = '2100'
pi_year  = 1865
ic_year  = 2086
doy      = 211 # day_of_year
## Change/Check these parameters before running
season     = 'SON'
season0    = 'son'
months     = [9,10,11]
day        =  350      #First day of the season
date       = 'Nov'

In [56]:
#To reduce communication between workers, I am manually fusing tasks
#dask.optimization.fuse(ave_width=50)

In [57]:
%%time
ds_ssp  = xr.open_mfdataset(cesm2_path+'ssp370_cesm2tmax.zarr',engine='zarr')
ds_hist = xr.open_mfdataset(cesm2_path+'bhist_cesm2tmax.zarr',engine='zarr')

CPU times: user 1.18 s, sys: 207 ms, total: 1.39 s
Wall time: 4.2 s


In [58]:
%%time
#t-temp, nw - no-warming, w-warming, ssn= season, g=global
tgpi_ssn   = ds_hist.TREFHTMX.sel(time=ds_hist.time.dt.month.isin(months)).\
         sel(time=slice(pi_year0,pi_year1))
tgeoc_ssn  = ds_ssp.TREFHTMX.sel(time=ds_ssp.time.dt.month.isin(months)).\
         sel(time=slice(ic_year0,ic_year1))

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 314 ms, sys: 17.2 ms, total: 331 ms
Wall time: 351 ms


In [59]:
tgpi_ssn   = tgpi_ssn.chunk({'member':100,'lat':40,'lon':40})
tgeoc_ssn  = tgeoc_ssn.chunk({'member':100,'lat':40,'lon':40})
tdgpi_ssn  = to_daily(tgpi_ssn)
tdgeoc_ssn = to_daily(tgeoc_ssn)

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/dataset.py:4880: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  result = result._unstack_full_reindex(
/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/dataset.py:4880: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

In [60]:
pcoeffs_gpi_ssn  = tdgpi_ssn.mean('member').polyfit(dim='year',deg=1)
pcoeffs_geoc_ssn = tdgeoc_ssn.mean('member').polyfit(dim='year',deg=1)
#pcoeffs_geoc_ssn

In [61]:
mgpi_ssn  = pcoeffs_gpi_ssn.polyfit_coefficients.sel(degree=1)
mgeoc_ssn = pcoeffs_geoc_ssn.polyfit_coefficients.sel(degree=1)
mgeoc_ssn 

<xarray.DataArray 'polyfit_coefficients' (lat: 192, lon: 288, day: 91)>
dask.array<getitem, shape=(192, 288, 91), dtype=float64, chunksize=(5, 288, 91), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * day      (day) int64 244 245 246 247 248 249 250 ... 329 330 331 332 333 334
    degree   int64 1

In [62]:
gpi_trend_ssn  = mgpi_ssn*(tdgpi_ssn['year']-pi_year)
geoc_trend_ssn = mgeoc_ssn*(tdgeoc_ssn['year']-ic_year)
gpi_trend_ssn

,Array,Chunk
Bytes,1.12 GiB,29.99 MiB
Shape,"(192, 288, 91, 30)","(5, 288, 91, 30)"
Count,7712 Tasks,40 Chunks
Type,float64,numpy.ndarray


In [63]:
tgpi_det_ssn  = tdgpi_ssn  - gpi_trend_ssn
tgeoc_det_ssn = tdgeoc_ssn - geoc_trend_ssn
tgpi_det_ssn  = tgpi_det_ssn.chunk({'day':10,'lat':5})
tgeoc_det_ssn = tgeoc_det_ssn.chunk({'day':10,'lat':5})
tgeoc_det_ssn

,Array,Chunk
Bytes,112.47 GiB,15.26 MiB
Shape,"(192, 288, 100, 30, 91)","(5, 40, 100, 10, 10)"
Count,40788 Tasks,12480 Chunks
Type,float64,numpy.ndarray


In [64]:
%%time
#Stack member and year into one dimension- 'ym'. Then, roll over this dimension with stride = N_mem.
#This works because of the way xarray stacks two dimensions. window_len =30. 
#I have called the 'window' dimension 'index'.
quants  = np.arange(0,1.01,0.01)
########################
N_mem       = 100 
window_len  = 30
#### July 30 #######
tgpi_detroll_ssn   = stack_roll(tgpi_det_ssn,N_mem*window_len,N_mem,stackdim_name='ym')
tgeoc_detroll_ssn  = stack_roll(tgeoc_det_ssn,N_mem*window_len,N_mem,stackdim_name='ym')
#########
tgpi_detroll0_ssn  = tgpi_detroll_ssn.sel(ym=[15])  
tgeoc_detroll0_ssn = tgeoc_detroll_ssn.sel(ym=[15])  
tgeoc_detroll0_ssn

CPU times: user 469 ms, sys: 12.1 ms, total: 481 ms
Wall time: 466 ms


,Array,Chunk
Bytes,112.47 GiB,45.78 MiB
Shape,"(192, 288, 91, 1, 3000)","(5, 40, 10, 1, 3000)"
Count,177149 Tasks,3120 Chunks
Type,float64,numpy.ndarray


In [65]:
%%time
############# Save files ########################
tgeoc_detroll_ssnpath  = cvals_det + 'tgeoc_detrended_ssnroll_'+season0+'_'+pi_year0+'_'+pi_year1+'.zarr'
# Save eoc detrended rolled array
tgeoc_detroll0_ssn     =  tgeoc_detroll0_ssn.rename('eoc_detrended').assign_coords(ym=('ym',[2086]))
tgeoc_detroll0_ssn.to_dataset().to_zarr(tgeoc_detroll_ssnpath,mode='w')

CPU times: user 10min 43s, sys: 24.7 s, total: 11min 8s
Wall time: 28min 22s


In [66]:
%%time
#Compute the pre-industrial + moments adjusted curves
#First compute rolling means and stdevs and save them
#Open
tgeoc_detroll_ssn   = xr.open_zarr(tgeoc_detroll_ssnpath).eoc_detrended
tgeoc_det_mroll_ssn = tgeoc_detroll_ssn.mean(dim='index')
tgeoc_det_sroll_ssn = tgeoc_detroll_ssn.std(dim='index')
#############
tgeoc_det_mroll_ssn

CPU times: user 2.05 s, sys: 63.5 ms, total: 2.12 s
Wall time: 2.08 s


<xarray.DataArray 'eoc_detrended' (lat: 192, lon: 288, day: 91, ym: 1)>
dask.array<mean_agg-aggregate, shape=(192, 288, 91, 1), dtype=float64, chunksize=(5, 40, 10, 1), chunktype=numpy.ndarray>
Coordinates:
  * day      (day) int64 244 245 246 247 248 249 250 ... 329 330 331 332 333 334
    degree   int64 ...
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member   (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    year     (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * ym       (ym) int64 2086

In [67]:
#Now prepare the pre-industrial data array in order to apply the moment delta mapping transformations
###Add a dummy dimension called 'ym' to tgpi_detrended after stacking year and member as 'myear'
############ det ###########
tgpi_det_stack_ssn = stacker(tgpi_det_ssn,'myear')
###################################
tgpi_det_stack0_ssn= tgpi_det_stack_ssn.expand_dims(dim={'ym':1}).\
                 assign_coords(ym=('ym',[2086]))
tgpi_det_stack0_ssn

,Array,Chunk
Bytes,112.47 GiB,15.26 MiB
Shape,"(1, 192, 288, 91, 3000)","(1, 5, 40, 10, 1000)"
Count,60832 Tasks,9360 Chunks
Type,float64,numpy.ndarray


In [68]:
%%time
#############
tgmc_det_ssn  = mean_adj(tgpi_det_stack0_ssn, tgeoc_det_mroll_ssn)
tgmsc_det_ssn = mean_std_adj(tgpi_det_stack0_ssn, tgeoc_det_mroll_ssn,tgeoc_det_sroll_ssn)
tgmc_det_ssn

CPU times: user 1.24 s, sys: 35.8 ms, total: 1.28 s
Wall time: 1.66 s


,Array,Chunk
Bytes,112.47 GiB,15.26 MiB
Shape,"(1, 192, 288, 91, 3000)","(1, 5, 40, 10, 1000)"
Count,113873 Tasks,9360 Chunks
Type,float64,numpy.ndarray


In [69]:
%%time
#### Compute quantiles 
quants        = np.arange(0,1.01,0.01)
qgeoc_det_ssn = tgeoc_detroll0_ssn.chunk(dict(index=-1)).quantile(quants,dim='index',skipna=False)
qgmc_det_ssn  = tgmc_det_ssn.chunk(dict(myear=-1)).quantile(quants,dim='myear',skipna=False)
qgmsc_det_ssn = tgmsc_det_ssn.chunk(dict(myear=-1)).quantile(quants,dim='myear',skipna=False)
qgeoc_det_ssn

CPU times: user 234 ms, sys: 5.92 ms, total: 240 ms
Wall time: 231 ms


<xarray.DataArray 'eoc_detrended' (quantile: 101, lat: 192, lon: 288, day: 91,
                                   ym: 1)>
dask.array<transpose, shape=(101, 192, 288, 91, 1), dtype=float64, chunksize=(101, 5, 40, 10, 1), chunktype=numpy.ndarray>
Coordinates:
    year      (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * day       (day) int64 244 245 246 247 248 249 ... 329 330 331 332 333 334
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member    (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * ym        (ym) int64 2086
  * quantile  (quantile) float64 0.0 0.01 0.02 0.03 0.04 ... 0.97 0.98 0.99 1.0

In [70]:
%%time
qgmsc_ssndetpath     = cvals_det+'qgmsc_percentile_det_ssn_'+season0+'_'+ic_year0+'_'+ic_year1+'.zarr'
qgmc_ssndetpath      = cvals_det+'qgmc_percentile_det_ssn_'+season0+'_' +ic_year0+'_'+ic_year1+'.zarr'
qgauss_ssndetpath    = cvals_det+'qgauss_percentile_det_ssn_'+season0+'_' +ic_year0+'_'+ic_year1+'.zarr'
qgaussmo_ssndetpath  = cvals_det+'qgaussmo_percentile_det_ssn_'+season0+'_' +ic_year0+'_'+ic_year1+'.zarr'
qgeoc_ssndetpath     = cvals_det+'qgeoc_percentile_det_ssn_'+season0+'_' +ic_year0+'_'+ic_year1+'.zarr'
print(qgmc_ssndetpath)
##################### Save #####################
qgmc_det_ssn    = qgmc_det_ssn.rename('qgmc_detrended').chunk({'lat':8})
qgmsc_det_ssn   = qgmsc_det_ssn.rename('qgmsc_detrended').chunk({'lat':8})
qgeoc_det_ssn   = qgeoc_det_ssn.rename('qgeoc_detrended').chunk({'lat':8})

/global/scratch/users/harsha/LENS/cesm2/cvals/detrended/qgmc_percentile_det_ssn_son_2071_2100.zarr
CPU times: user 211 ms, sys: 8.14 ms, total: 219 ms
Wall time: 211 ms


In [71]:
########## EOC gaussians ###########
tgeoc_det_mroll0_ssn = tgeoc_det_mroll_ssn.expand_dims(dim={'quantile':101}).\
               assign_coords(quantile=('quantile',quants))
tgeoc_det_sroll0_ssn = tgeoc_det_sroll_ssn.expand_dims(dim={'quantile':101}).\
               assign_coords(quantile=('quantile',quants))
# Create a DataArray for the quantiles
quantiles = xr.DataArray(np.arange(0, 1.01, 0.01), dims='quantile').\
assign_coords(quantile=('quantile',quants))
####### Construct gaussians for model warming ################
qg        = xr.apply_ufunc(erfinv,2*quantiles-1)
#################
qgauss_det_ssn      = tgeoc_det_mroll0_ssn + tgeoc_det_sroll0_ssn * np.sqrt(2) *qg
qgauss_det_ssn.name = 'qgauss_detrended'
qgauss_det_ssn
## gaussm = gaussian with eoc mean only and pre-ind std
qgaussmo_det_ssn      = tgeoc_det_mroll0_ssn + tgpi_det_stack0_ssn.std('myear')* np.sqrt(2) *qg       
qgaussmo_det_ssn.name = 'qgaussmo_detrended'
qgaussmo_det_ssn

<xarray.DataArray 'qgaussmo_detrended' (quantile: 101, lat: 192, lon: 288,
                                        day: 91, ym: 1)>
dask.array<add, shape=(101, 192, 288, 91, 1), dtype=float64, chunksize=(101, 5, 40, 10, 1), chunktype=numpy.ndarray>
Coordinates:
  * day       (day) int64 244 245 246 247 248 249 ... 329 330 331 332 333 334
    degree    int64 1
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member    (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    year      (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * ym        (ym) int64 2086
  * quantile  (quantile) float64 0.0 0.01 0.02 0.03 0.04 ... 0.97 0.98 0.99 1.0

In [ ]:
%%time
#Save files
qgauss_det_ssn.to_dataset().to_zarr(qgauss_ssndetpath,mode='w')
qgmc_det_ssn.to_dataset().to_zarr(qgmc_ssndetpath,mode='w') 
qgmsc_det_ssn.to_dataset().to_zarr(qgmsc_ssndetpath,mode='w')
qgeoc_det_ssn.to_dataset().to_zarr(qgeoc_ssndetpath,mode='w') 
qgaussmo_det_ssn.to_dataset().to_zarr(qgaussmo_ssndetpath,mode='w') 

In [ ]:
qgmc_det_ssn     = xr.open_zarr(qgmc_ssndetpath).qgmc_detrended
qgmsc_det_ssn    = xr.open_zarr(qgmsc_ssndetpath).qgmsc_detrended
qgauss_det_ssn   = xr.open_zarr(qgauss_ssndetpath).qgauss_detrended
qgeoc_det_ssn    = xr.open_zarr(qgeoc_ssndetpath).qgeoc_detrended
qgaussmo_det_ssn = xr.open_zarr(qgaussmo_ssndetpath).qgaussmo_detrended

In [ ]:
###### Error in prediction for EOC #######
qgmsc_detssn_err    = qgmsc_det_ssn.sel(ym=2086)  - qgeoc_det_ssn.sel(ym=2086)
qgmc_detssn_err     = qgmc_det_ssn.sel(ym=2086)   - qgeoc_det_ssn.sel(ym=2086)
qgauss_detssn_err   = qgauss_det_ssn.sel(ym=2086) - qgeoc_det_ssn.sel(ym=2086)
qgaussmo_detssn_err = qgaussmo_det_ssn.sel(ym=2086) - qgeoc_det_ssn.sel(ym=2086)
qgaussmo_detssn_err 

In [ ]:
%%time
########### Save the absolute errors ############
####
qgmsc_detssn_errpath    = cvals_det+'qgmsc_detssn_err_' +season0 +'_'+ic_year0+'_'+ic_year1+'.zarr'
qgmc_detssn_errpath     = cvals_det+'qgmc_detssn_err_'  +season0 +'_'+ic_year0+'_'+ic_year1+'.zarr'
qgauss_detssn_errpath   = cvals_det+'qgauss_detssn_err_'+season0 +'_'+ic_year0+'_'+ic_year1+'.zarr'
qgaussmo_detssn_errpath = cvals_det+'qgaussmo_detssn_err_'+season0 +'_'+ic_year0+'_'+ic_year1+'.zarr'
#### Save ########
print(qgmc_detssn_errpath)
qgmsc_detssn_err    = qgmsc_detssn_err.rename('Errors').chunk({'lat':20})
qgmc_detssn_err     = qgmc_detssn_err.rename('Errors').chunk({'lat':20})
qgauss_detssn_err   = qgauss_detssn_err.rename('Errors').chunk({'lat':20})
qgaussmo_detssn_err = qgaussmo_detssn_err.rename('Errors').chunk({'lat':20})
############
qgmsc_detssn_err.to_dataset().to_zarr(qgmsc_detssn_errpath,mode='w')
qgmc_detssn_err.to_dataset().to_zarr(qgmc_detssn_errpath,mode='w')
qgauss_detssn_err.to_dataset().to_zarr(qgauss_detssn_errpath,mode='w')
qgaussmo_detssn_err.to_dataset().to_zarr(qgaussmo_detssn_errpath,mode='w')

In [ ]:
######## New colorbar only for shift + stretch and gaussian ############
x = 5
# create a figure and axis
fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5)
# Define colormap
cmap = plt.get_cmap('RdBu_r')
# Make a norm object with the center at 0: TwoSlopeNorm
norm = mcolors.TwoSlopeNorm(vmin=-x, vcenter=0, vmax=x)
# Making numpy array from -3 to 3, with step 0.2
values = np.arange(-x, x+0.25, 0.25)
# Creating a mappable object and setting the norm and cmap for colorbar
mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
mappable.set_array([])
# Creating a colorbar
ticks1 = altspace(-4,1,9)
cbar = plt.colorbar(mappable, ax=ax, orientation='vertical',ticks=ticks1)
cbar.set_label('')
plt.gca().set_visible(False)

In [ ]:
%%time
# Creating a figure and axes
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5), subplot_kw={'projection': cart.crs.PlateCarree()}\
                       , gridspec_kw = {'wspace':0.05, 'hspace':0.2})

# Plotting qgmsc_err
qgmsc_detssn_err.sel(quantile=0.9).mean('day').plot(ax=axs[0], transform=cart.crs.PlateCarree(),\
                                                    add_colorbar=False, cmap=cmap, norm=norm)
axs[0].coastlines(color="black")
axs[0].set_title(season+r': $L_2(Q_i) - Q_f$ at $p=0.9$')

# Plotting da2
im = qgauss_detssn_err.sel(quantile=0.9).mean('day').plot(ax=axs[1], transform=cart.crs.PlateCarree(),\
                                                          add_colorbar=False, cmap=cmap, norm=norm)
axs[1].coastlines(color="black")
axs[1].set_title(season+': $G_2(Q_i) - Q_f$ at $p=0.9$')

# Adding colorbar
cbar = plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.5, orientation='vertical')
cbar.set_label('Errors')

plt.show()

In [ ]:
%%time
# Creating a figure and axes
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5), subplot_kw={'projection': cart.crs.PlateCarree()}\
                       , gridspec_kw = {'wspace':0.05, 'hspace':0.2})

# Plotting qgmsc_err
qgmc_detssn_err.sel(quantile=0.9).mean('day').plot(ax=axs[0], transform=cart.crs.PlateCarree(),\
                                                    add_colorbar=False, cmap=cmap, norm=norm)
axs[0].coastlines(color="black")
axs[0].set_title(season+': $L_1(Q_i) - Q_f$ at $p=0.9$')

# Plotting da2
im = qgaussmo_detssn_err.sel(quantile=0.9).mean('day').plot(ax=axs[1], transform=cart.crs.PlateCarree(),\
                                                          add_colorbar=False, cmap=cmap, norm=norm)
axs[1].coastlines(color="black")
axs[1].set_title(season+r': $G_1(Q_i) - Q_f$ at $p=0.9$')

# Adding colorbar
cbar = plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.5, orientation='vertical')
cbar.set_label('Errors')

plt.show()

In [ ]:
####### Coded until here ###########

In [ ]:
qgaussmo_detssn_err = xr.open_zarr(qgaussmo_detssn_errpath).Errors

In [ ]:
################################### Coded until here ############

In [ ]:
# %%time
# #Check if the results are reproduced for Bengaluru
# year      = np.arange(1865,2087)
# qmw       = qgmw.sel(lat=LAT,lon=LON,method='nearest')
# qnw       = qgnw.sel(lat=LAT,lon=LON,method='nearest')
# qmc       = qgmc.sel(lat=LAT,lon=LON,method='nearest')
# qmsc      = qgmsc.sel(lat=LAT,lon=LON,method='nearest')
# qgauss    = qgauss_mw.sel(lat=LAT,lon=LON,method='nearest')

# plt.plot(year,qmw.sel(quantile=0.9),'--',color='tab:blue',label='actual')
# plt.plot(year,qmsc.sel(quantile=0.9),color='tab:green',label='pi + shift + stretch')
# plt.plot(year,qgauss.sel(quantile=0.9),color='red',label='gaussian')
# # plt.plot(year,qnw.sel(quantile=0.9),color='tab:blue',label='pi')
# # plt.plot(year,qmc.sel(quantile=0.9),color='tab:orange',label='pi + shift')
# #########
# plt.plot(2086,qmw.sel(quantile=0.9,tm=2086),marker='o',markerfacecolor='tab:blue',\
#          markeredgecolor='tab:blue', lw=2, ms=3, mew=2, color='tab:blue')

# plt.plot(2086,qmsc.sel(quantile=0.9,tm=2086),marker='o',markerfacecolor='tab:green',\
#          markeredgecolor='tab:green', lw=2, ms=3, mew=2, color='tab:green')

# plt.plot(2086,qgauss.sel(quantile=0.9,tm=2086),marker='o',markerfacecolor='red',\
#          markeredgecolor='red', lw=2, ms=3, mew=2, color='red')

# # plt.plot(2086,qnw.sel(quantile=0.9,tm=2086),marker='o',markerfacecolor='tab:blue',\
#          # markeredgecolor='tab:blue', lw=2, ms=3, mew=2, color='tab:blue')

# # plt.plot(2086,qmc.sel(quantile=0.9,tm=2086),marker='o',markerfacecolor='tab:orange',\
#          # markeredgecolor='tab:orange', lw=2, ms=3, mew=2, color='tab:orange')


# plt.legend(loc='upper left')
# plt.title('CESM2: q=0.9,' +locn)
# plt.ylabel(date+': Tmax (K)')
# plt.xlabel('Time')
# #plt.xlim(1864,2087)
# plt.xticks(np.round(np.linspace(1865,2086,8,endpoint=True),0))
# plt.show()

In [ ]:
#Now, let us compute the relative changes 1) qgmw_diff -0 2) qgmw_diff -qgmc_diff 3) qgmw_diff - qgmsc_diff

In [ ]:
# fig, axes = plt.subplots(1, 3, figsize=(15, 5), subplot_kw={'projection': cart.crs.PlateCarree()})

# data_arrays = [qgmw_diff.sel(quantile=0.9), qgmw_reldiff1.sel(quantile=0.9), qgmw_reldiff2.sel(quantile=0.9)]
# titles = [' pi - actual', ' (pi + shift) - actual', '(pi + shift + stretch) - actual']
# locations = [(-87.629, 41.94, 'tab:orange'), (77.5, 12.72, 'tab:brown')]

# for ax, data, title in zip(axes, data_arrays, titles):
#     img = data.plot(ax=ax, transform=cart.crs.PlateCarree(), cmap=cmap, norm=norm, add_colorbar=False)
#     for loc in locations:
#         ax.plot(loc[0], loc[1], marker='*', color=loc[2])
#     ax.set_xlabel('lon')
#     ax.set_ylabel('lat')
#     ax.coastlines(color="black")
#     ax.set_title(title)

# # Add a single colorbar
# fig.subplots_adjust(right=0.8,wspace=0.1)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
# fig.colorbar(img, cax=cbar_ax, ticks=ticks)
# # Add a title to the whole plot
# fig.suptitle("July 30 Tmax, q=0.9", fontsize=15)

# plt.show()